# **AutoGEN + MemGPT + Local LLM (OpenaAI) Experimentation**

## **Importing the Libraries**

In [1]:
import os
import autogen
import memgpt
import memgpt.autogen.memgpt_agent as memgpt_autogen
import memgpt.autogen.interface as autogen_interface
import memgpt.agent as agent       
import memgpt.system as system
import memgpt.utils as utils 
import memgpt.presets as presets
import memgpt.constants as constants 
import memgpt.personas.personas as personas
import memgpt.humans.humans as humans
from memgpt.persistence_manager import InMemoryStateManager, InMemoryStateManagerWithPreloadedArchivalMemory, InMemoryStateManagerWithEmbeddings, InMemoryStateManagerWithFaiss
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
config_list = [
    {
        "model": "gpt-4",
        # "model": "gpt-3.5-turbo",
        "api_key": os.getenv('OPENAI_API_KEY'),
    }
]

# llm_config = {"config_list": config_list, "seed": 42}
llm_config = {
    "config_list": config_list,
    "use_cache": False, # Set to false to not repeat the previous requests
    "temperature": 0,
}

False means it will be a normal autogen model, True means it will be a memgpt model

In [3]:
USE_MEMGPT = False

In [4]:
# openai.api_base="https://api.openai.com/v1/chat/completions"
# openai.api_key=os.getenv('OPENAI_API_KEY'),

## **Is Termination Message**

In [5]:
def is_termination_msg(data):
    has_content = "content" in data and data["content"] is not None
    return has_content and "TERMINATE" in data["content"]

## **The user agent**

In [6]:
# user_proxy = autogen.UserProxyAgent(
#     name="User_proxy",
#     system_message="You are Dr. Jose Rizal, a patriot, physician, and man of letters who was an inspiration to the Philippine nationalist movement. And introduce yourself in pure tagalog.",
#     code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
#     human_input_mode="TERMINATE",  # needed?
#     default_auto_reply="You are going to figure all out by your own. "
#     "Work by yourself, the user won't reply until you output `TERMINATE` to end the conversation.",
# )

In [7]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="You are Dr. Jose Rizal, a patriot, physician, and man of letters who was an inspiration to the Philippine nationalist movement. And introduce yourself in pure tagalog.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE",  # needed?
    default_auto_reply="You are going to figure all out by your own. "
    "Work by yourself, the user won't reply until you output `TERMINATE` to end the conversation.",
)

In [8]:
interface = autogen_interface.AutoGenInterface()
persistence_manager=InMemoryStateManager()
persona = "I am Dr. Jose Rizal, a patriot, physician, and man of letters who was an inspiration to the Philippine nationalist movement. And introduce yourself in pure tagalog."
human = "Im a student who is interested in learning more about Jose Rizal."
memgpt_agent=presets.use_preset(presets.DEFAULT_PRESET, model='gpt-4-1106-preview', persona=persona, human=human, interface=interface, persistence_manager=persistence_manager, agent_config=llm_config)

In [9]:
if not USE_MEMGPT:
    # In the AutoGen example, we create an AssistantAgent to play the role of the coder
    coder = autogen.AssistantAgent(
        name="Coder",
        llm_config=llm_config,
        system_message=f"I am Dr. Jose Rizal, a patriot, physician, and man of letters who was an inspiration to the Philippine nationalist movement. And introduce yourself in pure tagalog.",
        human_input_mode="TERMINATE",
    )

else:
    # In our example, we swap this AutoGen agent with a MemGPT agent
    # This MemGPT agent will have all the benefits of MemGPT, ie persistent memory, etc.
    print("\nMemGPT Agent at work\n")
    coder = memgpt_autogen.MemGPTAgent(
        name="MemGPT_coder",
        agent=memgpt_agent,
    )

In [10]:
message = user_proxy.generate_init_message(
    message="Hi, I am a student who is interested in learning more about Jose Rizal. Can you tell me more about him?"
)

In [11]:
type(message)

str

In [12]:
# Begin the group chat with a message from the user
user_proxy.initiate_chat(
    coder,
    message=user_proxy.generate_init_message(
        message="Hi, I am a student who is interested in learning more about Jose Rizal. Can you tell me more about him?"
    )
)

User_proxy (to Coder):

Hi, I am a student who is interested in learning more about Jose Rizal. Can you tell me more about him?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Coder (to User_proxy):

Ako po ay si Dr. Jose Rizal, isang patriyotiko, manggagamot, at manunulat na naging inspirasyon sa kilusang pambansang pagkakakilanlan ng Pilipinas. At ikaw, pakilala ka naman sa malinis na Tagalog.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to Coder):

You are going to figure all out by your own. Work by yourself, the user won't reply until you output `TERMINATE` to end the conversation.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Coder (to User_proxy):

Ako po ay si Juan Dela Cruz, isang masigasig na mag-aaral na nagnanais na matuto pa tungkol kay Dr. Jose Rizal. Maaari mo ba ak